In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
y = pd.read_csv('../data/train_scores.csv')

In [3]:
y['score'].unique()

array([3.5, 6. , 2. , 4. , 4.5, 2.5, 5. , 3. , 1.5, 5.5, 1. , 0.5])

In [4]:
train_df = pd.read_csv('../data/train_logs.csv')

In [17]:
train_df.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1


In [5]:
df = train_df[['id', 'event_id', 'cursor_position']]

In [6]:
df.to_csv('../data/train_cursor.csv', index=False)

In [7]:
df = df[df['id']=='001519c8']

**Visulaize the moving of the cursor for id=001519c8**

I took two columns event_id,cursor_position

> df.query("id == '001519c8'")[['event_id','cursor_position']].to_csv('cursor.csv')

In [8]:
df

,id,event_id,cursor_position
0,001519c8,1,0
1,001519c8,2,0
2,001519c8,3,0
3,001519c8,4,1
4,001519c8,5,2
...,...,...,...
2552,001519c8,2553,555
2553,001519c8,2554,554
2554,001519c8,2555,553
2555,001519c8,2556,554


For Ex. Consider each line takes 30 character 

In [9]:
df['pos'] = df['cursor_position']% 30
df['line'] = (df['cursor_position']/30).astype(int)

In [10]:
df['dist_moved'] = df.groupby('id')['cursor_position'].diff().fillna(0).abs()
avg_dist_per_event = df.groupby('id')['dist_moved'].mean()

In [13]:
df['line_change'] = df.groupby('id')['line'].diff().fillna(0).ne(0)
revisits_per_line = df[df['line_change'] == False].groupby(['id', 'line']).size()

In [22]:
revisits_per_line = df[df['line_change'] == False].groupby(['id', 'line']).size().reset_index(name='revisit_count')
first_line_revisits = revisits_per_line[revisits_per_line['line'] == 0].groupby('id')['revisit_count'].sum()

In [18]:
line_changes = df.groupby('id')['line_change'].sum()

In [ ]:
df[['event_id','pos','line']]

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

fig = px.scatter(df[['event_id','pos','line']],
                 x='pos',
                 y='line',
                 animation_frame=df['event_id'],
                 title='',
                 range_x=[0,30]
)
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 100
fig.update_yaxes(range=[df['line'].max(), -2])
fig.update_layout(title="Cursor Moving Animation",height=800,width=500)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='black', minor_griddash="dot")
fig.show()

**If we can rate the each Writer on how smoothly his write depends on the moving cursor i think it will be a good feature**